# Reproduce core figures

This notebook demonstrates how to reconstruct the piecewise power‑law fit for the damping ratio and reproduce the corresponding figure.  It uses approximate Particle Data Group masses and widths for a set of reference particles.  Feel free to edit the data frame or the break mass to explore alternative fits.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Define approximate masses (GeV) and widths (GeV) for the reference particles.
data = pd.DataFrame({
    'particle': ['pi','K','p','tau','J/psi','Upsilon(1S)','W','Z','H','t'],
    'mass': [0.1396, 0.4937, 0.9383, 1.77686, 3.0969, 9.4603, 80.379, 91.1876, 125.25, 172.76],
    # Widths for pi and p are extremely small; we set the proton width to 1e-12 GeV to avoid zeros
    'width': [2.53e-17, 5.32e-14, 1e-12, 2.27e-12, 9.29e-5, 5.41e-5, 2.085, 2.495, 4.07e-3, 1.41]
})

# Compute damping ratio Gamma/m
data['damping'] = data['width'] / data['mass']

# Select a break mass (GeV) separating low and high regimes
break_mass = 106.9

# Perform linear fits in log–log space for each regime
def fit_power_law(x, y):
    # Fit log10(y) = slope * log10(x) + intercept
    res = linregress(np.log10(x), np.log10(y))
    return res.slope, res.intercept

masses = data['mass']
ratios = data['damping']
idx_low = masses <= break_mass
idx_high = masses > break_mass

slope_low, intercept_low = fit_power_law(masses[idx_low], ratios[idx_low])
slope_high, intercept_high = fit_power_law(masses[idx_high], ratios[idx_high])

print(f'Low‑mass slope: {slope_low:.3f}, intercept: {intercept_low:.3f}')
print(f'High‑mass slope: {slope_high:.3f}, intercept: {intercept_high:.3f}')

# Plot damping ratio vs mass with piecewise fits
plt.figure(figsize=(6,4))
colors = ['blue' if m <= break_mass else 'red' for m in masses]
plt.scatter(masses, ratios, c=colors, edgecolor='k')

# Define x‑ranges for the fits
x1 = np.logspace(np.log10(masses[idx_low].min()), np.log10(break_mass), 100)
x2 = np.logspace(np.log10(masses[idx_high].min()), np.log10(masses[idx_high].max()), 100)

plt.plot(x1, 10**intercept_low * x1**slope_low, '--', color='blue')
plt.plot(x2, 10**intercept_high * x2**slope_high, '--', color='red')

plt.axvline(break_mass, color='gray', linestyle=':')
plt.xscale('log'); plt.yscale('log')
plt.xlabel('mass (GeV)')
plt.ylabel('damping ratio (Gamma/m)')
plt.title('Damping ratio vs mass with piecewise fits')
plt.tight_layout()
plt.show()
